In [ ]:
#input must be options chain day1 - end

In [23]:
import os,h5py
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler

from IPython.display import clear_output
import joblib
import random
import hashlib

In [24]:
CSV_PATH     = './data/OptionsEOD.csv/'
#create @ part 1
PARQUET_PATH = './data/OptionsEOD.parquet'
#create @ part 2
PARQUET_STG_PATH = './data/OptionsEOD_STG.parquet'
SCALER_COL  = ['UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','TOTAL_VOLUME','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
#create @ part 3
ID = ['OPTIONS_ID']
SCALER_PATH = ['./data/scaler/' , 'scaler.gz']
SCALER_PRICE_PATH = ['./data/scaler/' , 'price_scaler.gz']
SCALER_PRICE_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in ['C_BID',	'C_ASK',  'P_BID',	'P_ASK'] ]
SCALER_OTHER_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v not in ['C_BID',	'C_ASK',  'P_BID',	'P_ASK'] ]
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
H5_PATH = './data/OptTrainData/'
START = True#True#'2010-09'#True


max_option_len = 16

In [25]:
#==== Create options_qoute
options_qoute = {}
#==== Create MinMaxScaler
if not os.path.exists(SCALER_PATH[0]):
    os.makedirs(SCALER_PATH[0])

In [26]:
#example
EOD_CSV = pd.read_csv(CSV_PATH+"qqq/qqq_eod_201201.txt", engine='pyarrow')
EOD_CSV.head()

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_LAST],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT]
0,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.003130,0.002320,0.000350,-0.004080,0.000000,0.793590,,10.9,0.192
1,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.003580,0.002780,0.000530,-0.004420,-0.000420,0.721270,,9.9,0.174
2,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.004040,0.003340,0.000950,-0.004350,-0.000430,0.648640,,8.9,0.156
3,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.010000,-0.004390,0.004150,0.001170,-0.004050,-0.000330,0.577770,0.000000,7.9,0.139
4,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,0.971540,0.016360,...,0.010000,-0.004860,0.005270,0.001370,-0.004140,0.000000,0.507560,0.000000,6.9,0.121


In [ ]:
==================================Create OptionsEOD.parquet part 1========================================

In [ ]:
#Part I
#TransformData : 
#-each partition from EXPIRE_DATE 
#-csv too parquet
#-col. rename 
def TransformDataI():
    #scaler = MinMaxScaler()
    scaler = StandardScaler()
    price_scaler = StandardScaler()
    schema = None
    pqwriter = None
    for d in os.listdir(CSV_PATH):
        for f in os.listdir(CSV_PATH+f"{d}/"):
            if f.endswith(".txt"):
                ## load
                print( f"[LOAD] : {CSV_PATH}{d}/{f}        ",end='\r')
                EOD_CSV = pd.read_csv(CSV_PATH+f"{d}/"+f, engine='pyarrow')
                    
                ## rename col.
                for c in EOD_CSV.columns:
                    EOD_CSV = EOD_CSV.rename( columns={ c:c.strip().replace(']','').replace('[','') } )
                
                ## add symbol 
                EOD_CSV['SYMBOL'] = d.upper()
                ## add INTRINSIC_VALUE
                EOD_CSV['INTRINSIC_VALUE'] = EOD_CSV['UNDERLYING_LAST'] - EOD_CSV['STRIKE']
                
                ## fillnafillna
                EOD_CSV['P_VOLUME'] = EOD_CSV['P_VOLUME'].fillna(0)
                EOD_CSV['C_VOLUME'] = EOD_CSV['C_VOLUME'].fillna(0)
                EOD_CSV.dropna(subset=[SCALER_COL])
            
                
                # date columns convert to datetime
                for c in ["QUOTE_READTIME","QUOTE_DATE","EXPIRE_DATE"]:
                    EOD_CSV[c] = pd.to_datetime(EOD_CSV[c])
                
                #clean float data
                for c in ['INTRINSIC_VALUE','C_DELTA','C_GAMMA','C_VEGA','C_THETA','C_RHO','C_IV','C_VOLUME','C_LAST','C_BID','C_ASK','STRIKE','P_BID','P_ASK','P_LAST','P_DELTA','P_GAMMA','P_VEGA','P_THETA','P_RHO','P_IV','P_VOLUME','STRIKE_DISTANCE','STRIKE_DISTANCE_PCT']:
                    if EOD_CSV[c].dtype not in ( 'float32','float64'):
                        EOD_CSV[c] = EOD_CSV[c].apply(lambda x: x.strip())
                        EOD_CSV[c] = EOD_CSV[c].replace('', np.nan).fillna(np.nan)
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                    if EOD_CSV[c].dtype == 'float32':
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                        
                # REMAIN_DAYS(int) =>  use DTE col.
                #partition with QUOTE_DATE
                EOD_CSV['PartitionDate'] = EOD_CSV['QUOTE_DATE'].dt.strftime('%Y-%m')
                EOD_CSV.sort_values(['QUOTE_DATE','EXPIRE_DATE','SYMBOL','STRIKE'],ascending =False ) 

                #scaler(Normalization_
                #scaler.partial_fit(EOD_CSV[SCALER_COL])

                # save
                if os.path.exists(PARQUET_PATH):
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
                else:
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
                    
    # joblib.dump(scaler, SCALER_PATH )
    # if pqwriter:
    #     pqwriter.close()
    # print( f"[DONE]                                                       ",end='\r')



In [ ]:
##-RunCleanData
#TransformDataI()

In [ ]:
==================================Create OptTrainData STG part 2========================================

In [ ]:
##### Part II 
#START = '2012-02'
#TransformData : 
# - read each partitions 
# - Normalization if not have scaler.gz file
###===== load Scaler
if START == True:
    Scaler = StandardScaler()
    Scaler_Price = StandardScaler()
else:
    Scaler = joblib.load(''.join(SCALER_PATH ) )
    Scaler_Price = joblib.load(''.join(SCALER_PRICE_PATH ) )
###===== local functions
def strikeZero(df,v,num_rm):   
    # First, filter based on QUOTE_DATE, SYMBOL, and EXPIRE_DATE
    filtered_arr = df[(df['QUOTE_DATE'] == v['QUOTE_DATE']) &
                     (df['SYMBOL'] == v['SYMBOL']) &
                     (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])]['STRIKE'].values
    
    if num_rm != 0 :
        df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
                & (df['SYMBOL'] == v['SYMBOL']) 
                & (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])
                & (  (df['STRIKE'].isin(filtered_arr[:num_rm])  )
                   | (df['STRIKE'].isin(filtered_arr[-num_rm:])   )
                  ) 
            , SCALER_COL    
        ] = 1e-8
    
def hash_str(S):
    return hashlib.md5(S.encode('utf-8')).hexdigest() 
    
#========================================= main model =========================================
keys = None#df[unique_keys].sort_values(by=unique_keys).drop_duplicates()

#==== Get PartitionDate
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
#PartitionDate = ['2011-12','2022-05'] # debug
for i,partdate in enumerate(PartitionDate[:]) :  
    START = True if START == partdate else START
    if START == True :
        df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                     , filters=[('PartitionDate', '=', partdate)]
                                    )
        df['PartitionDate'] = df['PartitionDate'].astype('object') 
        #add col options_id
        df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
        df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
        df[df == 0] = 1e-8
        df = df.dropna(subset=[c for c in SCALER_COL if c != 'TOTAL_VOLUME' ])
        ####################################################
        keys = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()[:]
        DATA  = np.empty((0,) + ( max_option_len,len(SCALER_COL) ) ) #init 
        #loop each UNIQUE_KEYS keys 
        for j,v in keys.iterrows():
            msg = {}
            msg['qd_ate'] = v['QUOTE_DATE']
            msg['symbo'] = v['SYMBOL']
            msg['exdate'] = v['EXPIRE_DATE']
            
            df_filter=df[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) & (df['SYMBOL'] == v['SYMBOL']) & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) ]
            msg['start_rows'] = len(df_filter) 

            #break loops
            if len(df_filter) < 5 : 
                print(f"""{msg['qd_ate']}|{msg['symbo']}|{msg['exdate']} : {msg['start_rows']}->break (less rows)""")
                break 

            qoute = "".join(v[ ['SYMBOL','EXPIRE_DATE'] ].apply(str).values)
            #add new qoute
            if qoute not in [*options_qoute.keys()]:
                options_qoute[qoute] = {}
                options_qoute[qoute]['start_price'] = df_filter['UNDERLYING_LAST'].values[0]
                options_qoute[qoute]['strike'] = df_filter[ df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]) ]['STRIKE'].values
                options_qoute[qoute]['exp'] = df_filter['EXPIRE_DATE'].values[0]
                #check diff UNDERLYING_LAST
                if df_filter['UNDERLYING_LAST'].values[0] != round(np.average(df_filter['UNDERLYING_LAST']),4):
                    print('[ERROR] : set UNDERLYING_LAST ',qoute )
            #rm index max : max_option_len
            #may be initstrike out of scope
            rm_strike_index = df_filter[ ~df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ].index
            df_filter = df_filter.drop(rm_strike_index)

            #break loops
            if len(df_filter) < 5 :  
                print(f"""{msg['qd_ate']}|{msg['symbo']}|{msg['exdate']} : {msg['start_rows']}->break (rm_strike_index)""")
                break
                
            # Generate zero strike 
            # Generate a random float between 0.01 and 1
            
            dte = df_filter[ df_filter['DTE'] != 1e-8]['DTE'].mean()
            df_filter['DTE'] = dte
            random_number = random.uniform(0, 1)
            nom_rm_rows = [r  for r in [0.1,0.2,0.3,0.4] if random_number < r]

            strikeZero(df_filter,v,len(nom_rm_rows) )
            df_filter.loc[df_filter['DTE'] != 0,'TOTAL_VOLUME'] = df_filter[  df_filter['DTE'] != 0  ]['P_VOLUME'].sum() + df_filter[  df_filter['DTE'] != 0  ]['C_VOLUME'].sum()
            df_filter.loc[df_filter['DTE'] == 1e-8,'TOTAL_VOLUME'] = 1e-8
            hash_id = hash_str(f'{i}{j}')
            
            # #++++++++++++++++++++++++++++++++++++++++++++++
            # print( "A")
            # print( len(df_filter) )
            # print( nom_rm_rows )
            # print( df_filter['STRIKE_DISTANCE'] )
            # break
            # #++++++++++++++++++++++++++++++++++++++++++++++
            
            #====filed rows
            z = 0
            while len(df_filter) != max_option_len :
                z += 1
                if z > 50: raise "error"
                Even= -1
                if (len(df_filter) % 2) == 0:
                    Even = 0
                if len(df_filter) < max_option_len :
                    zero_row = pd.DataFrame([[1e-8]*len(df_filter.columns)], columns=df_filter.columns)
                    if Even==0 :
                        df_filter = pd.concat([df_filter, zero_row], ignore_index=True)
                    else:
                        df_filter = pd.concat([zero_row, df_filter], ignore_index=True)
                elif len(df_filter) > max_option_len :
                     df_filter = df_filter.drop(df_filter.index[Even])
            
            #add cal columns.      
            msg['end_rows']= len(df_filter)
            if len(df_filter) == max_option_len and not np.isnan( np.sum( df_filter[SCALER_COL].values ) ) :
                DATA = np.vstack((DATA ,[df_filter[SCALER_COL]]))
            print(f"""{msg['qd_ate']}|{msg['symbo']}|{msg['exdate']} : {msg['start_rows']}->{msg['end_rows']}  """)
        if len(DATA) :
            clear_output(wait = False)
            #========================== partial_fit ================================#
            Scaler.partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_OTHER_COL_INDEX] )
            Scaler_Price.partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_PRICE_COL_INDEX].reshape(-1, len(SCALER_PRICE_COL_INDEX)) )
            joblib.dump(Scaler, "".join( ''.join(SCALER_PATH ) ) )
            joblib.dump(Scaler_Price, "".join( ''.join(SCALER_PRICE_PATH ) ) )
            #=== clear expire options_qoute
            for qi in list(options_qoute.keys()):
                if options_qoute[qi]['exp'] < keys['QUOTE_DATE'].drop_duplicates().values.max():
                    options_qoute.pop(qi)
            
            #========================== SAVE DATA ================================#
            #=== save H5
            if not os.path.exists(H5_PATH):
                os.makedirs(H5_PATH)
            with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
                dset = f.create_dataset(f'{partdate}', data=DATA, chunks=True , compression='gzip')
            # #=== save parquet
            # if os.path.exists(PARQUET_STG_PATH):
            #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
            # else:
            #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )   
        
            print(f"[Last Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%       ")
            print("---------------------------------------------------------------")

[Last Processing] 2012-07, 4.24%       
---------------------------------------------------------------
2012-08-01 00:00:00|QQQ|2012-08-03 00:00:00 : 22->16  
2012-08-01 00:00:00|QQQ|2012-08-17 00:00:00 : 50->16  
2012-08-01 00:00:00|QQQ|2012-09-21 00:00:00 : 55->16  
2012-08-01 00:00:00|QQQ|2012-09-28 00:00:00 : 40->16  
2012-08-01 00:00:00|QQQ|2012-10-19 00:00:00 : 52->16  
2012-08-01 00:00:00|QQQ|2012-11-16 00:00:00 : 47->16  
2012-08-01 00:00:00|QQQ|2012-12-21 00:00:00 : 58->16  
2012-08-01 00:00:00|QQQ|2012-12-31 00:00:00 : 33->16  
2012-08-01 00:00:00|QQQ|2013-01-18 00:00:00 : 61->16  
2012-08-01 00:00:00|QQQ|2013-03-15 00:00:00 : 55->16  
2012-08-01 00:00:00|QQQ|2013-03-28 00:00:00 : 42->16  
2012-08-01 00:00:00|QQQ|2013-06-28 00:00:00 : 41->16  
2012-08-01 00:00:00|QQQ|2014-01-17 00:00:00 : 29->16  
2012-08-01 00:00:00|QQQ|2014-12-19 00:00:00 : 13->16  
2012-08-01 00:00:00|SPX|2012-08-03 00:00:00 : 71->16  
2012-08-01 00:00:00|SPX|2012-08-10 00:00:00 : 70->16  
2012-08-01 00:00

In [ ]:
==========================

In [ ]:
Scaler.partial_fit( DATA.reshape(-1, len(SCALER_COL)) )

In [ ]:
DATA.reshape(-1, len(SCALER_COL)).shape

In [ ]:
len(SCALER_COL)

In [ ]:
DATA.shape

In [ ]:
DATA

In [ ]:
pd.DataFrame(  
    DATA[0]
,columns=SCALER_COL)


In [ ]:
++++++ PartIII to H5 ++++++ 

In [ ]:
#Example
import random
from IPython.display import clear_output,display, HTML
#load scaler
scaler = MinMaxScaler()
if not os.path.exists(H5_PATH):
    os.makedirs(H5_PATH)
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
random.shuffle(PartitionDate)
scaler = joblib.load( ''.join(SCALER_PATH ) )
pd.options.mode.copy_on_write = True

for i,partdate in enumerate(PartitionDate) :
    df = pd.read_parquet(PARQUET_STG_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
    df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
    DATA  = np.empty((0,) + (20,9) ) 
    for opt_id in np.unique( df[["OPTIONS_ID"]].values):
        df_filter  = df[df["OPTIONS_ID"]==opt_id]

        if len(df_filter) > 15 :
            while len(df_filter) != 20:
                Even= -1
                if (len(df_filter) % 2) == 0:
                    Even = 0
                if len(df_filter) < 20 :
                    df_filter.loc[len(df_filter) ] = [0]*len(df_filter.columns)
                    total_vol = df_filter['TOTAL_VOLUME'].values.mean()
                    df_filter.loc[:, 'TOTAL_VOLUME'] = total_vol
                    df_filter.loc[:, 'OPTIONS_ID']= opt_id
                elif len(df_filter) > 20 :
                     df_filter = df_filter.drop(df_filter.index[Even])

        
        if len(df_filter) == 20 and not np.isnan( np.sum( df_filter[SCALER_COL].values ) ) :
            DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))
                    
    # Save the NumPy array to an HDF5 file
    with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
        dset = f.create_dataset(f'{partdate}', data=DATA, chunks=True , compression='gzip')

    print(f"[Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')

In [ ]:
============== sampling show====================

In [ ]:
=========================================== TEST ==============================================

In [ ]:
import os
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
for partdate in PartitionDate[:1] :
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    print(f"CONFIG['transform'] : {CONFIG['transform']}")

    DATA = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
    DATA = DATA.reshape(data_shape)

In [ ]:
import numpy as np
a=np.array([[1,2,3],
[4,5,6]
])
a[:,-1]

In [ ]:
a.shape